# Tutorial 04 - Graetz

In [ ]:
import typing

In [ ]:
import dolfinx.fem
import dolfinx.io
import gmsh
import mpi4py.MPI
import multiphenicsx.io
import numpy as np
import numpy.typing
import petsc4py.PETSc
import ufl

In [ ]:
import rbnicsx.backends

## 1. Mesh generation

In [ ]:
mesh_size = 5e-2

In [ ]:
gmsh.initialize()
gmsh.model.add("graetz")

In [ ]:
p0 = gmsh.model.geo.addPoint(0.0, 0.0, 0.0, mesh_size)
p1 = gmsh.model.geo.addPoint(1.0, 0.0, 0.0, mesh_size)
p2 = gmsh.model.geo.addPoint(1.0, 1.0, 0.0, mesh_size)
p3 = gmsh.model.geo.addPoint(0.0, 1.0, 0.0, mesh_size)
l0 = gmsh.model.geo.addLine(p0, p1)
l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p0)
left_loop = gmsh.model.geo.addCurveLoop([l0, l1, l2, l3])
left_subdomain = gmsh.model.geo.addPlaneSurface([left_loop])

In [ ]:
p4 = gmsh.model.geo.addPoint(2.0, 0.0, 0.0, mesh_size)
p5 = gmsh.model.geo.addPoint(2.0, 1.0, 0.0, mesh_size)
l4 = gmsh.model.geo.addLine(p1, p4)
l5 = gmsh.model.geo.addLine(p4, p5)
l6 = gmsh.model.geo.addLine(p5, p2)
right_loop = gmsh.model.geo.addCurveLoop([l4, l5, l6, -l1])
right_subdomain = gmsh.model.geo.addPlaneSurface([right_loop])

In [ ]:
gmsh.model.geo.synchronize()
gmsh.model.addPhysicalGroup(1, [l0], 1)
gmsh.model.addPhysicalGroup(1, [l4], 2)
gmsh.model.addPhysicalGroup(1, [l5], 3)
gmsh.model.addPhysicalGroup(1, [l6], 4)
gmsh.model.addPhysicalGroup(1, [l2], 5)
gmsh.model.addPhysicalGroup(1, [l3], 6)
gmsh.model.addPhysicalGroup(2, [left_subdomain], 1)
gmsh.model.addPhysicalGroup(2, [right_subdomain], 2)
gmsh.model.mesh.generate(2)

In [ ]:
mesh, subdomains, boundaries = dolfinx.io.gmshio.model_to_mesh(
    gmsh.model, comm=mpi4py.MPI.COMM_WORLD, rank=0, gdim=2)
gmsh.finalize()

In [ ]:
multiphenicsx.io.plot_mesh(mesh)

In [ ]:
multiphenicsx.io.plot_mesh_tags(subdomains)

In [ ]:
multiphenicsx.io.plot_mesh_tags(boundaries)

## 2. Problem definition

In [ ]:
class AffineShapeParametrization(rbnicsx.backends.MeshMotion):
    """Deform the domain with an affine shape parametrization."""

    def __init__(self, mu: np.typing.NDArray[np.float64]) -> None:
        # Define function space
        M = dolfinx.fem.VectorFunctionSpace(mesh, ("Lagrange", mesh.geometry.cmap.degree))
        # Interpolate affine shape parametrization expression on a dolfinx Function
        shape_parametrization = dolfinx.fem.Function(M)
        shape_parametrization.interpolate(
            lambda x: (x[0], x[1]), subdomains.indices[subdomains.values == 1])
        shape_parametrization.interpolate(
            lambda x: (mu[0] * (x[0] - 1) + 1, x[1]), subdomains.indices[subdomains.values == 2])
        # Initialize mesh motion object
        super().__init__(mesh, shape_parametrization)

In [ ]:
mu_mesh_motion = np.array([10.0, np.nan])
with AffineShapeParametrization(mu_mesh_motion):
    multiphenicsx.io.plot_mesh_tags(subdomains)

In [ ]:
class Problem(object):
    """Define a linear problem, and solve it with KSP."""

    def __init__(self) -> None:
        # Define function space
        V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 1))
        self._V = V
        # Define trial and test functions
        u = ufl.TrialFunction(V)
        v = ufl.TestFunction(V)
        # Define solution
        solution = dolfinx.fem.Function(V)
        self._solution = solution
        # Define symbolic parameters for use in UFL forms
        mu_symb = rbnicsx.backends.SymbolicParameters(mesh, shape=(2, ))
        self._mu_symb = mu_symb
        # Define bilinear form of the problem
        x = ufl.SpatialCoordinate(mesh)
        beta = ufl.as_vector((x[1] * (1 - x[1]), 0))
        a = (
            mu_symb[1] * ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
            + ufl.inner(ufl.dot(beta, ufl.grad(u)), v) * ufl.dx
        )
        self._a = a
        self._a_cpp = dolfinx.fem.form(a)
        # Define linear form of the problem
        zero = dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(0))
        f = ufl.inner(zero, v) * ufl.dx
        self._f = f
        self._f_cpp = dolfinx.fem.form(f)
        # Define boundary conditions for the problem
        one = dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(1))
        facets_zero = boundaries.indices[np.isin(boundaries.values, (1, 5, 6))]
        facets_one = boundaries.indices[np.isin(boundaries.values, (2, 4))]
        bdofs_V_zero = dolfinx.fem.locate_dofs_topological(V, mesh.topology.dim - 1, facets_zero)
        bdofs_V_one = dolfinx.fem.locate_dofs_topological(V, mesh.topology.dim - 1, facets_one)
        bcs = [dolfinx.fem.dirichletbc(zero, bdofs_V_zero, V), dolfinx.fem.dirichletbc(one, bdofs_V_one, V)]
        self._bcs = bcs
        # Prepare storage for mesh motion object
        self._mesh_motion: typing.Optional[rbnicsx.backends.MeshMotion] = None

    @property
    def function_space(self) -> dolfinx.fem.FunctionSpace:
        """Return the function space of the problem."""
        return self._V

    @property
    def bilinear_form(self) -> ufl.Form:  # type: ignore[no-any-unimported]
        """Return the bilinear form of the problem."""
        return self._a

    @property
    def linear_form(self) -> ufl.Form:  # type: ignore[no-any-unimported]
        """Return the linear form of the problem."""
        return self._f

    @property
    def boundary_conditions(self) -> typing.List[dolfinx.fem.DirichletBCMetaClass]:
        """Return the boundary conditions for the problem."""
        return self._bcs

    @property
    def mesh_motion(self) -> rbnicsx.backends.MeshMotion:
        """Return the mesh motion object that was used in the latest solve."""
        assert self._mesh_motion is not None
        return self._mesh_motion

    def _assemble_matrix(self) -> petsc4py.PETSc.Mat:  # type: ignore[no-any-unimported]
        """Assemble the left-hand side matrix."""
        A = dolfinx.fem.petsc.assemble_matrix(self._a_cpp, bcs=self._bcs)
        A.assemble()
        return A

    def _assemble_vector(self) -> petsc4py.PETSc.Vec:  # type: ignore[no-any-unimported]
        """Assemble the right-hand side vector."""
        F = dolfinx.fem.petsc.assemble_vector(self._f_cpp)
        dolfinx.fem.petsc.apply_lifting(F, [self._a_cpp], [self._bcs])
        F.ghostUpdate(addv=petsc4py.PETSc.InsertMode.ADD, mode=petsc4py.PETSc.ScatterMode.REVERSE)
        dolfinx.fem.petsc.set_bc(F, self._bcs)
        return F

    def solve(self, mu: np.typing.NDArray[np.float64]) -> dolfinx.fem.Function:
        """Assign the provided parameters value, apply shape parametrization and solve the problem."""
        self._mu_symb.value[:] = mu
        with AffineShapeParametrization(mu) as self._mesh_motion:
            return self._solve()

    def _solve(self) -> dolfinx.fem.Function:
        """Solve the linear problem with KSP."""
        A = self._assemble_matrix()
        F = self._assemble_vector()
        ksp = petsc4py.PETSc.KSP()
        ksp.create(mesh.comm)
        ksp.setOperators(A)
        ksp.setType("preonly")
        ksp.getPC().setType("lu")
        ksp.getPC().setFactorSolverType("mumps")
        ksp.setFromOptions()
        solution = self._solution.copy()
        ksp.solve(F, solution.vector)
        solution.vector.ghostUpdate(
            addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)
        return solution

In [ ]:
problem = Problem()

In [ ]:
mu_solve = np.array([10.0, 0.01])
solution = problem.solve(mu_solve)

In [ ]:
with problem.mesh_motion:
    multiphenicsx.io.plot_scalar_field(solution, "high fidelity solution")